In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
pd.set_option("display.max.columns", None)

In [ ]:
# TSC Value obtained with: sudo bpftrace -e 'BEGIN { printf("%u\n", *kaddr("tsc_khz")); exit(); }
TSC={}
TSC["don"]=2599950 
# time_in_nanoseconds = (tsc_count_end - tsc_count_start) * tsc_frequency
results = {}

# time in seconds
def time(value,m):
    return (value * TSC[m])/1000000000

def processData(WORK=["NULLLOOP", "NOPLOOP", "4NOPLOOP"], 
                CPU0S=[3,7], CPU1S=[17,46], 
                MACHINES=["don"], 
                DATES=["14-11-22:15:25:37"], 
                TYPES=["LOCAL_WORK", "LOCAL_WORK_NO_SERVERTHREAD","LOCAL_WORK_WITH_REMOTE_READ_SHARED",
                       "LOCAL_WORK_WITH_REMOTE_RW_SHARED","REMOTE_WORK_READ_SHARED","REMOTE_WORK_RW_SHARED"],
                COUNTS=[100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600, 51200],
                VERBOSE=False):
    for W in WORK:
        for CPU0 in CPU0S:
            for CPU1 in CPU1S:
                for MACHINE in MACHINES:
                    for DATE in DATES:
                        for TYPE in TYPES:
                            KEY=MACHINE+"-"+str(CPU0)+","+str(CPU1)+"-"+W+"-"+TYPE
                            results[KEY] =  pd.DataFrame(columns=['count','ctime','wtime','cerr','werr'])
                            for COUNT in COUNTS:
                                FILE="data/"+MACHINE+"/"+DATE+"/bm."+W+"_"+TYPE+"_"+str(COUNT)+"_"+str(CPU0)+"_"+str(CPU1)+".times"
                                if VERBOSE:
                                    print("processing:" + FILE)
                                data=pd.read_csv(FILE)
                                count=data['count'][0]
                                cpu0=data['cpu0'][0]
                                cpu1=data['cpu1'][0]
                                data['pctime']=time(data['ctime']/count, MACHINE)
                                data['pwtime']=time(data['wtime']/count, MACHINE)
                                minCtime=time(data['pctime'].min(), MACHINE)
                                maxCtime=time(data['pctime'].max(), MACHINE)
                                meanCtime=time(data['pctime'].mean(), MACHINE)
                                errCtime=np.array([meanCtime - minCtime, maxCtime - meanCtime])
                                minWtime=time(data['pwtime'].min(),MACHINE)
                                maxWtime=time(data['pwtime'].max(),MACHINE)
                                meanWtime=time(data['pwtime'].mean(),MACHINE)
                                errWtime=np.array([meanWtime - minWtime, maxWtime - meanWtime])
                                newres = pd.DataFrame([{'count':count, 'ctime':meanCtime, 'wtime':meanWtime, 'cerr':errCtime, 'werr':errWtime}])
                                results[KEY] = pd.concat([results[KEY], newres], axis=0,ignore_index=True)

def plot(KEYS, loff=0):
    plt.figure()
    fig, ax = plt.subplots()
    for key in KEYS:
#    ax.set_ylim(bottom=0)
        ax.errorbar(results[key]["count"],results[key]["ctime"],
                    yerr=np.array(results[key]['cerr'].tolist()).T,
                    label=key+"_CTIME", marker='x')
        ax.errorbar(results[key]["count"],results[key]["wtime"],
                    yerr=np.array(results[key]['werr'].tolist()).T,
                    label=key+"_WTIME", marker='o', linestyle="dotted")
#    plt.ylim(bottom=0)
    # ax.set_yscale('log')
    ax.set(xlabel='work count', ylabel='time in seconds')
    plt.legend(bbox_to_anchor=(0,1.4+loff), loc='upper center', ncol=1)
    plt.show()

In [ ]:
processData(COUNTS=[3200, 6400, 12800, 25600, 51200])


In [ ]:
plot(loff=.6, KEYS=['don-3,17-NOPLOOP-LOCAL_WORK', 'don-3,46-NOPLOOP-LOCAL_WORK', 'don-7,17-NOPLOOP-LOCAL_WORK', 'don-7,46-NOPLOOP-LOCAL_WORK'])

In [ ]:
plot(KEYS=['don-3,17-4NOPLOOP-LOCAL_WORK'])

In [ ]:
plot(KEYS=['don-3,17-NULLLOOP-LOCAL_WORK', 'don-3,17-NOPLOOP-LOCAL_WORK', 'don-3,17-4NOPLOOP-LOCAL_WORK'])

In [ ]:
plot(['don-3,17-NOPLOOP-LOCAL_WORK','don-3,17-NOPLOOP-LOCAL_WORK_NO_SERVERTHREAD'])

In [ ]:
plot(['don-3,17-NOPLOOP-LOCAL_WORK', 'don-3,17-NOPLOOP-LOCAL_WORK_WITH_REMOTE_READ_SHARED'])

In [ ]:
plot(['don-3,17-NOPLOOP-LOCAL_WORK', 'don-3,17-NOPLOOP-REMOTE_WORK_READ_SHARED'])